IMPORT

In [1]:
import numpy as np
import pandas as pd
import re

from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE

from bokeh.io import curdoc, push_notebook, output_notebook
from bokeh.layouts import column, layout
from bokeh.models import ColumnDataSource, Div, Select, Slider, TextInput, HoverTool
from bokeh.plotting import figure, show
from ipywidgets import interact, interactive, fixed, interact_manual

Loading data

In [53]:
data = pd.read_csv('C:/Users/isabe/OneDrive/Ambiente de Trabalho/project module 2 final/skincare_products_clean.csv')
data

product_name  \
0     The Ordinary Natural Moisturising Factors + HA...   
1         CeraVe Facial Moisturising Lotion SPF 25 52ml   
2     The Ordinary Hyaluronic Acid 2% + B5 Hydration...   
3             AMELIORATE Transforming Body Lotion 200ml   
4                        CeraVe Moisturising Cream 454g   
...                                                 ...   
1133  Elemis Life Elixirs Embrace Bath and Shower El...   
1134     Love Boo Splendidly Soothing Bath Soak (250ml)   
1135  Elemis Life Elixirs Fortitude Bath and Shower ...   
1136  Connock London Kukui Oil Soothing Bath & Showe...   
1137           Weleda Baby Calendula Cream Bath (200ml)   

                                            product_url product_type  \
0     https://www.lookfantastic.com/the-ordinary-nat...  Moisturiser   
1     https://www.lookfantastic.com/cerave-facial-mo...  Moisturiser   
2     https://www.lookfantastic.com/the-ordinary-hya...  Moisturiser   
3     https://www.lookfantastic.com/ameliorate-trans...  Moisturiser   
4     https://www.lookfantastic.com/cerave-moisturis...  Moisturiser   
...                                                 ...          ...   
1133  https://www.lookfantastic.com/elemis-life-elix...     Bath Oil   
1134  https://www.lookfantastic.com/love-boo-splendi...     Bath Oil   
1135  https://www.lookfantastic.com/elemis-life-elix...     Bath Oil   
1136  https://www.lookfantastic.com/connock-london-k...     Bath Oil   
1137  https://www.lookfantastic.com/weleda-baby-cale...     Bath Oil   

                                          clean_ingreds   price  
0     ['capric triglyceride', 'cetyl alcohol', 'prop...   £5.20  
1     ['homosalate', 'glycerin', 'octocrylene', 'eth...  £13.00  
2     ['sodium hyaluronate', 'sodium hyaluronate', '...   £6.20  
3     ['ammonium lactate', 'c12-15', 'glycerin', 'pr...  £22.50  
4     ['glycerin', 'cetearyl alcohol', 'capric trigl...  £16.00  
...                                                 ...     ...  
1133  ['prunus amygdalus dulcis', 'tipa-laureth sulf...  £55.00  
1134  ['sodium lauroyl', 'sodium cocoamphoacetate', ...  £10.99  
1135  ['prunus amygdalus dulcis', 'tipa-laureth sulf...  £55.00  
1136  ['capric triglyceride', 'peg-40 sorbitan perol...  £36.00  
1137  ['prunus amygdalus dulcis', 'sesamium indicum ...  £13.95  

[1138 rows x 5 columns]

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1138 entries, 0 to 1137
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_name   1138 non-null   object
 1   product_url    1138 non-null   object
 2   product_type   1138 non-null   object
 3   clean_ingreds  1138 non-null   object
 4   price          1138 non-null   object
dtypes: object(5)
memory usage: 44.6+ KB


There are no missing values and the ingredients column was previously thoroughly cleaned. All that remains is for the ingredients to be processed into a different format that can be used in the recommendation engine. One-hot encoding will be used.

In [5]:
for i in range(len(data['clean_ingreds'])):
    data['clean_ingreds'].iloc[i] = str(data['clean_ingreds'].iloc[i]).replace('[', '').replace(']', '').replace("'", '').replace('"', '')

C:\Users\isabe\AppData\Local\Temp\ipykernel_6316\3602423551.py:2: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['clean_ingreds'].iloc[i] = str(data['clean_ingreds'].iloc[i]).replace('[', '').replace(']', '').replace("'", '').replace('"'

In [6]:
all_ingreds = []

for i in data['clean_ingreds']:
    ingreds_list = i.split(', ')
    for j in ingreds_list:
        all_ingreds.append(j)

In [7]:
all_ingreds = sorted(set(all_ingreds))
all_ingreds[0:20]

['',
 '1,10-decanediol',
 '1,2-hexanediol',
 '1,2-hexanediol ',
 '1-methylhydantoin-2-imide',
 '10-hydroxydecanoic acid',
 '2,6-dimethyl-7-octen-2-ol',
 '2-bromo-2-nitropropane-1,3-diol',
 '2-oleamido-1',
 '3-o-ethyl ascorbic acid',
 '3-octadecanediol',
 '4-t-butylcyclohexanol',
 '7-dehydrocholesterol',
 'abies alba leaf oil',
 'abies balsamea extract',
 'abies sibirica oil',
 'acacia concinna fruit extract',
 'acacia decurrens wax',
 'acacia senegal gum',
 'acacia seyal gum extract']

In [8]:
all_ingreds.remove('')
for i in range(len(all_ingreds)):
    if all_ingreds[i][-1] == ' ':
        all_ingreds[i] = all_ingreds[i][0:-1]
        
all_ingreds = sorted(set(all_ingreds))
all_ingreds[0:20]

['1,10-decanediol',
 '1,2-hexanediol',
 '1-methylhydantoin-2-imide',
 '10-hydroxydecanoic acid',
 '2,6-dimethyl-7-octen-2-ol',
 '2-bromo-2-nitropropane-1,3-diol',
 '2-oleamido-1',
 '3-o-ethyl ascorbic acid',
 '3-octadecanediol',
 '4-t-butylcyclohexanol',
 '7-dehydrocholesterol',
 'abies alba leaf oil',
 'abies balsamea extract',
 'abies sibirica oil',
 'acacia concinna fruit extract',
 'acacia decurrens wax',
 'acacia senegal gum',
 'acacia seyal gum extract',
 'acer saccharum extract',
 'acetate']

In [ ]:
one_hot_list = [[0] * 0 for i in range(len(all_ingreds))]

for i in data['clean_ingreds']:
    k=0
    for j in all_ingreds:
        if j in i:
            one_hot_list[k].append(1)
        else:
            one_hot_list[k].append(0)
        k+=1
        
ingred_matrix = pd.DataFrame(one_hot_list).transpose()
ingred_matrix.columns = [sorted(set(all_ingreds))]

ingred_matrix

1,10-decanediol 1,2-hexanediol 1-methylhydantoin-2-imide  \
0                  0              0                         0   
1                  0              0                         0   
2                  0              0                         0   
3                  0              0                         0   
4                  0              0                         0   
...              ...            ...                       ...   
1133               0              0                         0   
1134               0              0                         0   
1135               0              0                         0   
1136               0              0                         0   
1137               0              0                         0   

     10-hydroxydecanoic acid 2,6-dimethyl-7-octen-2-ol  \
0                          0                         0   
1                          0                         0   
2                          0                         0   
3                          0                         0   
4                          0                         0   
...                      ...                       ...   
1133                       0                         0   
1134                       0                         0   
1135                       0                         0   
1136                       0                         0   
1137                       0                         0   

     2-bromo-2-nitropropane-1,3-diol 2-oleamido-1 3-o-ethyl ascorbic acid  \
0                                  0            0                       0   
1                                  0            0                       0   
2                                  0            0                       0   
3                                  0            0                       0   
4                                  0            0                       0   
...                              ...          ...                     ...   
1133                               0            0                       0   
1134                               0            0                       0   
1135                               0            0                       0   
1136                               0            0                       0   
1137                               0            0                       0   

     3-octadecanediol 4-t-butylcyclohexanol  ... zinc glycine zinc laurate  \
0                   0                     0  ...            0            0   
1                   0                     0  ...            0            0   
2                   0                     0  ...            0            0   
3                   0                     0  ...            0            0   
4                   0                     0  ...            0            0   
...               ...                   ...  ...          ...          ...   
1133                0                     0  ...            0            0   
1134                0                     0  ...            0            0   
1135                0                     0  ...            0            0   
1136                0                     0  ...            0            0   
1137                0                     0  ...            0            0   

     zinc oxide zinc pca zinc sulfate zingiber aromaticus extract  \
0             0        0            0                           0   
1             0        0            0                           0   
2             0        0            0                           0   
3             0        0            0                           0   
4             0        0            0                           0   
...         ...      ...          ...                         ...   
1133          0        0            0                           0   
1134          0        0            0                           0   
1135          0        0            0       

Extracting brand names

Many of the products in the dataset are from the same brands and almost share all of the same ingredients. It makes sense to use the brand name as a filter in the function to generate product recommendations from a good variety of brands and give the user more options (in the full project this was ONE OF the options - the user could choose to include products from the same brand if they wish).

Below is a list of all the brands included on the website that the data was taken from. We will use this list to extract the brand names from the product names. All product names contain the product's brand name.

In [11]:
brand_list = ["111skin", "a'kin", "acorelle", "adam revolution", "aesop", "ahava", "alchimie forever",
             "algenist", "alpha-h", "ambre solaire", "ameliorate", "american crew", "anthony", "antipodes",
             "apivita", "argentum", "ark skincare", "armani", "aromatherapy associates", "aromaworks", "aromatica",
             "aurelia probiotic skincare", "aurelia skincare",
             "australian bodycare", "avant skincare", "aveda", "aveeno", "avene", "avène",
             "bakel", "balance me", "barber pro", "bareminerals", "barry m cosmetics",
             "baxter of california", "bbb london", "beautypro", "benefit", "benton", "bioderma",
             "bioeffect", "bloom & blossom", "bloom and blossom", "bobbi brown", "bondi sands", "bubble t", "bulldog", "burt's bees",
             "by terry", "carita", "caudalie", "cerave", "chantecaille", "clinique",
             "comfort zone", "connock london", "cosmetics 27", "cosrx", "cowshed", "crystal clear", 
             "cult51", "darphin", "dear, klairs", "decleor", "decléor", "dermalogica", "dhc", "doctors formula",
             "dr. brandt", "dr brandt", "dr. hauschka", "dr hauschka", "dr. jackson's", "dr.jart+", "dr. lipp",
             "dr botanicals", "dr dennis", "dr. pawpaw", "ecooking", "egyptian magic",
             "eisenberg", "elemental herbology", "elemis", "elizabeth arden", "embryolisse",
             "emma hardie", "erno laszlo", "espa", "estée lauder", "estee lauder", "eucerin",
             "eve lom", "eve rebirth", "fade out", "farmacy", "filorga", "first aid beauty", "fit", "foreo",
             "frank body", "freezeframe", "gallinée", "garnier", "gatineau", "glamglow", "goldfaden md",
             "green people", "hawkins and brimble", "holika holika", "house 99", "huxley",
             "ilapothecary", "ila-spa", "indeed labs", "inika", "instant effects", "institut esthederm", "ioma", "klorane",
             "j.one", "jack black", "james read", "jason", "jo malone london", "juice beauty", "jurlique",
             "korres", "l:a bruket", "l'oréal men expert", "l'oreal men expert", "l'oréal paris", "l'oreal paris",
             "l’oréal paris", "lab series skincare for men",
             "lancaster", "lancer skincare", "lancôme", "lancome", "lanolips", "la roche-posay", "laura mercier",
             "liftlab", "little butterfly london", "lixirskin", "liz earle", "love boo",
             "löwengrip", "lowengrip", "lumene", "mac", "madara", "mádara", "magicstripes", "magnitone london",
             "mama mio", "mancave", "manuka doctor", "mauli", "mavala", "maybelline", "medik8", "men-u", "menaji", "molton brown", "moroccanoil",
             "monu", "murad", "naobay", "nars", "natio", "natura bissé", "natura bisse",
             "neal's yard remedies", "neom", "neostrata", "neutrogena", "niod", "nip+fab", "nuxe", "nyx",
             "oh k!", "omorovicza", "origins", "ortigia fico", "oskia", "ouai", "pai ", "paula's choice", "payot",
             "perricone md", "pestle & mortar", "pestle and mortar", "peter thomas roth",
             "philosophy", "pierre fabre", "pixi", "piz buin", "polaar", "prai", "project lip",
             "radical skincare", "rapideye", "rapidlash", "real chemistry", "recipe for men",
             "ren ", "renu", "revolution beauty", "revolution skincare", "rituals", "rmk", "rodial", "roger&gallet", "salcura",
             "sanctuary spa", "sanoflore", "sarah chapman", "sea magik", "sepai",
             "shaveworks", "shea moisture", "shiseido", "skin79", "skin authority", "skinceuticals",
             "skinchemists", "skindoctors", "skin doctors", "skinny tan", "sol de janeiro", "spa magik organiks",
              "st. tropez", "starskin", "strivectin", "sukin",
             "svr", "swiss clinic", "talika", "tan-luxe", "tanorganic", "tanworx", "thalgo", "the chemistry brand",
             "the hero project", "the inkey list", "the jojoba company", "the ordinary",
             "the organic pharmacy", "the ritual of namasté", "this works", "too faced", "trilogy", "triumph and disaster",
             "ultrasun", "uppercut deluxe", "urban decay", "uriage", "verso", "vichy",
             "vida glow", "vita liberata", "wahl", "weleda", "westlab", "wilma schumann", "yes to",
             "ysl", "zelens"]
brand_list = sorted(brand_list, key=len, reverse=True)

In [12]:
data['brand'] = data['product_name'].str.lower()
k=0
for i in data['brand']:
    for j in brand_list:
        if j in i:
            data['brand'][k] = data['brand'][k].replace(i, j.title())
    k+=1
    
data

C:\Users\isabe\AppData\Local\Temp\ipykernel_6316\1541820683.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['brand'][k] = data['brand'][k].replace(i, j.title())
C:\Users\isabe\AppData\Local\Temp\ipykernel_6316\1541820683.py:6: Setti

product_name  \
0     The Ordinary Natural Moisturising Factors + HA...   
1         CeraVe Facial Moisturising Lotion SPF 25 52ml   
2     The Ordinary Hyaluronic Acid 2% + B5 Hydration...   
3             AMELIORATE Transforming Body Lotion 200ml   
4                        CeraVe Moisturising Cream 454g   
...                                                 ...   
1133  Elemis Life Elixirs Embrace Bath and Shower El...   
1134     Love Boo Splendidly Soothing Bath Soak (250ml)   
1135  Elemis Life Elixirs Fortitude Bath and Shower ...   
1136  Connock London Kukui Oil Soothing Bath & Showe...   
1137           Weleda Baby Calendula Cream Bath (200ml)   

                                            product_url product_type  \
0     https://www.lookfantastic.com/the-ordinary-nat...  Moisturiser   
1     https://www.lookfantastic.com/cerave-facial-mo...  Moisturiser   
2     https://www.lookfantastic.com/the-ordinary-hya...  Moisturiser   
3     https://www.lookfantastic.com/ameliorate-trans...  Moisturiser   
4     https://www.lookfantastic.com/cerave-moisturis...  Moisturiser   
...                                                 ...          ...   
1133  https://www.lookfantastic.com/elemis-life-elix...     Bath Oil   
1134  https://www.lookfantastic.com/love-boo-splendi...     Bath Oil   
1135  https://www.lookfantastic.com/elemis-life-elix...     Bath Oil   
1136  https://www.lookfantastic.com/connock-london-k...     Bath Oil   
1137  https://www.lookfantastic.com/weleda-baby-cale...     Bath Oil   

                                          clean_ingreds   price            X  \
0     capric triglyceride, cetyl alcohol, propanedio...   £5.20   -54.835403   
1     homosalate, glycerin, octocrylene, ethylhexyl,...  £13.00   605.959839   
2     sodium hyaluronate, sodium hyaluronate, panthe...   £6.20   589.979675   
3     ammonium lactate, c12-15, glycerin, prunus amy...  £22.50   489.877350   
4     glycerin, cetearyl alcohol, capric triglycerid...  £16.00   616.318909   
...                                                 ...     ...          ...   
1133  prunus amygdalus dulcis, tipa-laureth sulfate,...  £55.00 -1160.279663   
1134  sodium lauroyl, sodium cocoamphoacetate, cocam...  £10.99  -427.265015   
1135  prunus amygdalus dulcis, tipa-laureth sulfate,...  £55.00 -1180.714844   
1136  capric triglyceride, peg-40 sorbitan peroleate...  £36.00  -573.088501   
1137  prunus amygdalus dulcis, sesamium indicum seed...  £13.95  -657.908875   

                Y           brand  
0     -972.399536    The Ordinary  
1      908.576355          Cerave  
2     -627.570740    The Ordinary  
3      678.181152      Ameliorate  
4     1050.192261          Cerave  
...           ...             ...  
1133  -372.142944          Elemis  
1134   702.305725        Love Boo  
1135  -378.035278          Elemis  
1136    95.726883  Connock London  
1137   134.687973          Weleda  

[1138 rows x 8 columns]

In [13]:
sorted(data.brand.unique())

["A'Kin",
 'Acorelle',
 'Aesop',
 'Ahava',
 'Alchimie Forever',
 'Alpha-H',
 'Ambre Solaire',
 'Ameliorate',
 'Antipodes',
 'Apivita',
 'Ark Skincare',
 'Armani',
 'Aromatherapy Associates',
 'Aromaworks',
 'Aurelia Probiotic Skincare',
 'Aurelia Skincare',
 'Australian Bodycare',
 'Avant Skincare',
 'Aveda',
 'Aveeno',
 'Avene',
 'Avène',
 'Balance Me',
 'Barber Pro',
 'Bareminerals',
 'Bbb London',
 'Beautypro',
 'Benefit',
 'Benton',
 'Bioderma',
 'Bloom & Blossom',
 'Bloom And Blossom',
 'Bobbi Brown',
 'Bondi Sands',
 'Bubble T',
 'Bulldog',
 "Burt'S Bees",
 'By Terry',
 'Caudalie',
 'Cerave',
 'Chantecaille',
 'Clinique',
 'Comfort Zone',
 'Connock London',
 'Cosrx',
 'Cowshed',
 'Crystal Clear',
 'Darphin',
 'Dear, Klairs',
 'Decléor',
 'Dermalogica',
 'Dhc',
 'Dr Brandt',
 'Dr Dennis',
 'Dr Hauschka',
 'Dr. Brandt',
 'Dr. Hauschka',
 'Dr. Pawpaw',
 'Dr.Jart+',
 'Egyptian Magic',
 'Elemental Herbology',
 'Elemis',
 'Elizabeth Arden',
 'Embryolisse',
 'Emma Hardie',
 'Erno Laszlo

There are slight variations in some of the brand names which would cause them to be considered as separate brands. This needs to be resolved before continuing or else the brand filter wont function correctly.

In [14]:
data['brand'] = data['brand'].replace(['Aurelia Probiotic Skincare'],'Aurelia Skincare')
data['brand'] = data['brand'].replace(['Avene'],'Avène')
data['brand'] = data['brand'].replace(['Bloom And Blossom'],'Bloom & Blossom')
data['brand'] = data['brand'].replace(['Dr Brandt'],'Dr. Brandt')
data['brand'] = data['brand'].replace(['Dr Hauschka'],'Dr. Hauschka')
data['brand'] = data['brand'].replace(["L'oreal Paris", 'L’oréal Paris'], "L'oréal Paris")

creating and adding a column with skin types

In [18]:
def map_skin_type(row):
    benefits = str(row.get('benefits', '')).lower()  # Certifique-se de que 'benefits' é uma string
    if 'oily' in benefits:
        return 'oily'
    elif 'dry' in benefits:
        return 'dry'
    elif 'sensitive' in benefits:
        return 'sensitive'
    elif 'normal' in benefits:
        return 'normal'
    else:
        return 'all'

In [19]:
data['skin_type'] = data.apply(map_skin_type, axis=1)

In [20]:
print(data['skin_type'].value_counts())

skin_type
all    1138
Name: count, dtype: int64


In [21]:
skin_types = ['oily', 'dry', 'combination', 'sensitive', 'normal']
for skin in skin_types:
    data[f'is_{skin}'] = data['skin_type'].apply(
        lambda x: 1 if isinstance(x, str) and skin in x else 0
    )

In [22]:
features = pd.concat([ingred_matrix, data[[f'is_{skin}' for skin in skin_types]]], axis=1)


Training Model

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import ast

# Load the data
data = pd.read_csv("updated_skincare_data_with_skin_types.csv")

#necessary columns are present
if 'ingredients_list' not in data.columns or 'skin_type' not in data.columns:
    raise KeyError("Columns 'ingredients_list' or 'skin_type' are missing from the dataset.")

# Pre-process the ingredients
def parse_ingredient_list(row):
    try:
        if isinstance(row, str):
            return [ingredient.strip().lower() for ingredient in ast.literal_eval(row)]
        return []
    except (ValueError, SyntaxError):
        return []

data['parsed_ingredients'] = data['ingredients_list'].apply(parse_ingredient_list)

#ingredient matrix using CountVectorizer
vectorizer = CountVectorizer(tokenizer=lambda x: x, preprocessor=lambda x: x)
ingredient_matrix = vectorizer.fit_transform(data['parsed_ingredients']).toarray()

# Add the ingredient matrix as a DataFrame
ingredient_matrix_df = pd.DataFrame(ingredient_matrix, columns=vectorizer.get_feature_names_out())

# Add binary columns for skin types
for skin in ['oily', 'dry', 'combination', 'sensitive', 'normal']:
    data[f"is_{skin}"] = data['skin_type'].apply(lambda x: 1 if x.lower() == skin else 0)

# Concatenate the data
features = pd.concat([ingredient_matrix_df, data[[f"is_{skin}" for skin in ['oily', 'dry', 'combination', 'sensitive', 'normal']]]], axis=1)

# Save the feature matrix for reuse
features.to_csv("ingredient_features.csv", index=False)
print("Ingredient feature matrix saved successfully.")

C:\Users\isabe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Ingredient feature matrix saved successfully.


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Load the feature data
features = pd.read_csv("ingredient_features.csv")

# Define skin types
skin_types = ['oily', 'dry', 'combination', 'sensitive', 'normal']

# Train a model for each skin type
models = {}
for skin in skin_types:
    print(f"Training model for {skin}...")

    # Define target and predictor variables
    target = features[f"is_{skin}"]
    X = features.drop(columns=[f"is_{skin}" for skin in skin_types])  # Remove all target columns

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

    # Train the model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for {skin}: {accuracy:.2f}")

    # Save the trained model
    joblib.dump(model, f"model_{skin}.pkl")
    models[skin] = model

print("All models trained and saved successfully.")

Training model for oily...
Accuracy for oily: 0.55
Training model for dry...
Accuracy for dry: 0.84
Training model for combination...
Accuracy for combination: 1.00
Training model for sensitive...
Accuracy for sensitive: 1.00
Training model for normal...
Accuracy for normal: 1.00
All models trained and saved successfully.


In [51]:
# Adjusting training to ensure feature consistency
all_ingredients = sorted(list(all_ingredients))  # Sort ingredients for consistency
features = pd.DataFrame(feature_matrix, columns=all_ingredients)

# Adding binary columns for skin types
for skin in skin_types:
    features[f"is_{skin}"] = data[f"is_{skin}"]

for skin in skin_types:
    # Define the target variable
    target = features[f"is_{skin}"]
    X = features.drop(columns=[f"is_{s}" for s in skin_types])  # Exclude target columns

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=42)

    # Train the model
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)

    joblib.dump((model, list(X.columns)), f"model_{skin}.pkl")


In [42]:
data = pd.read_csv("updated_skincare_data_with_skin_types.csv")
data['ingredients_list'] = data['ingredients_list'].dropna()
ingredient_list = list(set(ingredient.strip().lower() for group in data['ingredients_list'] if pd.notna(group) for ingredient in eval(group)))


In [44]:
print(ingredient_list)

['', 'cedrus atlantica bark oil', 'dodecane', 'taurine', 'melaleuca viridiflora leaf oil', 'creatine', 'fusanus spicatus wood oil', 'ergothioneine', 'galactomyces ferment filtrate', 'scabiosa arvensis extract', 'camellia kissi seed oil', 'jasminum grandiflorum flower extract', 'allantoin', 'diamond', 'sodium stearoyl lactylate', 'ceramide ns', 'cocamidopropyl hydroxysultaine', 'coriandrum sativum extract', 'dihydroxypropyl arganine hcl', 'aloe arborescens leaf extract', 'tapioca starch', 'methyl methacrylate/peg/ppg-4/3 methacrylate crosspolymer', 'palmitoyl grapevine extract', 'vaccinium corymbosum (blueberry) seed oil', 'cetraria islandica thallus extract', 'disteareth-75 ipdi', 'copper sulfate', 'lauryl peg/ppg-18/18 methicone', 'microcrystalline wax', 'polysorbate 60', 'lysine', 'isododecane', 'ethylhexyl cocoate', 'anhydroxylitol', 'niacinamide', 'folic acid', 'sodium butyroyl hyaluronate', 'glycyrrhetinic acid', 'polyglyceryl-3 pca', 'sodium dehydroacetate', 'rosa hybrid flower e

In [43]:
print(data['ingredients_list'].head())

0    ['glyceryl stearate', 'dimethicon', 'peg-8', '...
1                                                  NaN
2                                                  NaN
3                                                  NaN
4                                                  NaN
Name: ingredients_list, dtype: object
